In [1]:
import numpy as np
import networkx as nx
import random
import torch
from train import Train_Model
#from tqdm import tqdm, trange
from tqdm import tqdm_notebook as tqdm
import gc
from model import *
from utils import *

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

1.7.0
True
2


In [2]:
def pre_sample_for_ether(walk_times,adj_sparse, train_index, batch_size, save_name, do_walk):
    if do_walk:
        nodes_num = len(train_index)
        walks = torch.zeros(nodes_num, walk_times+1).long().cuda()
        batches = create_batches_forList(train_index, batch_size, True)
        i=0
        degrees = torch.tensor(adj_sparse.sum(1)).view(-1).cuda()
        candi_node = 0
        for batch in batches:
            walks[i*batch_size : i*batch_size + len(batch), 0] = torch.tensor(batch).cuda()
            candi_node = sparse_mx_to_torch_sparse_tensor(adj_sparse[batch.cpu().numpy()]).cuda()
            chosen_node = torch.zeros(len(batch), adj_sparse.shape[0]).cuda()
            for id in range(len(batch)):
                chosen_node[id][batch[id]] = 1.
            candi_node = ((- chosen_node + candi_node)> 0.0).float()
            for walk_id in range(walk_times):
                for x in range(candi_node.shape[0]):
                    if candi_node[x].sum()==0:
                        candi_node[x][batch[x]] = 1.
               
                p = candi_node# * degrees
                new_node = torch.multinomial(p,1).squeeze(1)
                walks[i*batch_size : i*batch_size + len(batch), walk_id+1] = new_node
                for id in range(len(batch)):
                    chosen_node[id][new_node[id]] = 1.
                candi_node = candi_node - chosen_node + sparse_mx_to_torch_sparse_tensor(adj_sparse[new_node.cpu()]).cuda()
                candi_node = (candi_node> 0.0).float()
                
            i+=1
        torch.cuda.empty_cache()
        result1 = np.array(walks.cpu())
        io.savemat(save_name+'.mat',{save_name:result1})
        return walks
    else:
        walks = io.loadmat(save_name+'.mat')
        print("walks",walks)
        return torch.tensor(walks[save_name]).cuda()

In [4]:
from sklearn.ensemble import *
from sklearn.metrics import accuracy_score

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
sparse_adj, sparse_adj_train,sparse_adj_train_all,features, train_feature,train_feature_all,labels, train_labels, id_train, id_valid, id_test, num_labels = Origin_load_ether_data()
sparse_adj.setdiag(1.0) 
sparse_adj_train.setdiag(1.0) 
sparse_adj_train_all.setdiag(1.0) 

train_mask = np.array(np.zeros(features.shape[0]), dtype=np.bool)
val_mask = np.array(np.zeros(features.shape[0]), dtype=np.bool)
test_mask = np.array(np.zeros(features.shape[0]), dtype=np.bool)
id_train = np.array(id_train)
id_valid = np.array(id_valid)
id_test = np.array(id_test)
train_mask[id_train] = True
val_mask[id_valid] = True
test_mask[id_test] = True
feat = features.cpu().numpy()
_labels = labels.cpu().numpy()
X_train, X_val, X_test = feat[train_mask], feat[val_mask], feat[test_mask]
y_train, y_val, y_test = _labels[train_mask], _labels[val_mask], _labels[test_mask]

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
acc = accuracy_score(y_test, clf.predict(X_test))
print("RandomForest accuracy: {}".format(acc))

features = (1402220, 4) type = <class 'numpy.ndarray'>
train_index 572
val_index 82
test_index 162
adj torch.Size([1402220, 1402220])
features torch.Size([1402220, 4])


/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


RandomForest accuracy: 0.9629629629629629


In [4]:
for walk in range(1,20):
    for seed in range(1, 20):
        torch.manual_seed(seed)
        random.seed(seed)
        torch.cuda.manual_seed(seed)
        np.random.seed(seed)
        save_name = 'seed='+str(seed)+'_walk='+str(walk)
        print('process...'+'train_'+save_name)
        pre_sample_for_ether(walk,sparse_adj_train_all, id_train, 32, './walks_ether/train_'+save_name, True)
        print('process...'+'valid_'+save_name)
        pre_sample_for_ether(walk,sparse_adj, id_valid, 32, './walks_ether/valid_'+save_name, True)
        print('process...'+'test_'+save_name)
        pre_sample_for_ether(walk,sparse_adj, id_test, 32, './walks_ether/test_'+save_name, True)

process...train_seed=1_walk=1


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


process...valid_seed=1_walk=1
process...test_seed=1_walk=1
process...train_seed=2_walk=1
process...valid_seed=2_walk=1
process...test_seed=2_walk=1
process...train_seed=3_walk=1
process...valid_seed=3_walk=1
process...test_seed=3_walk=1
process...train_seed=4_walk=1
process...valid_seed=4_walk=1
process...test_seed=4_walk=1
process...train_seed=5_walk=1
process...valid_seed=5_walk=1
process...test_seed=5_walk=1
process...train_seed=6_walk=1
process...valid_seed=6_walk=1
process...test_seed=6_walk=1
process...train_seed=7_walk=1
process...valid_seed=7_walk=1
process...test_seed=7_walk=1
process...train_seed=8_walk=1
process...valid_seed=8_walk=1
process...test_seed=8_walk=1
process...train_seed=9_walk=1
process...valid_seed=9_walk=1
process...test_seed=9_walk=1
process...train_seed=10_walk=1
process...valid_seed=10_walk=1
process...test_seed=10_walk=1
process...train_seed=11_walk=1
process...valid_seed=11_walk=1
process...test_seed=11_walk=1
process...train_seed=12_walk=1
process...vali

KeyboardInterrupt: 